In [ ]:
# 匯入套件
from tmdbv3api import TMDb
from tmdbv3api import Movie
import pandas as pd
import numpy as np
import requests
import json, os, pprint, time, re
from bs4 import BeautifulSoup as bs
from datetime import date
import matplotlib.pyplot as plt
import seaborn as sns

# 讀取json
df = pd.read_json('comments.json')

# 透過庭豪的篩選code資料清整
# 'NaN','','0',0,轉成''

for column_name in df.columns:
    df.loc[df[column_name] == 'NaN', column_name] = ''
    df.loc[df[column_name] == '' , column_name] = ''
    df.loc[df[column_name] == '0', column_name] = ''
    df.loc[df[column_name] == 0 , column_name] = ''
    
# 'title' , 'en_title' 篩掉標點符號 留英文 並將結果合併至 'title_sum_reg_new'

df['title_reg_new'] = df['title'].str.replace('[^a-zA-Z0-9 ]',' ',regex = True).str.replace(' +',' ',regex = True).str.extract('([ \dA-Za-z]*)')
df['title_sum_reg_new'] = df['en_title'].str.replace('[^a-zA-Z0-9 ]',' ',regex = True).str.replace(' +',' ',regex = True).str.extract('([ \dA-Za-z]*)')
df.loc[df['title_reg_new'] == ' ','title_reg_new'] = ''
df.loc[df['title_sum_reg_new'] == ' ','title_sum_reg_new'] = ''
df.loc[df['title_sum_reg_new'] == '','title_sum_reg_new'] = df.loc[df['title_sum_reg_new'] == '','title_reg_new']

# 如果是欄位是list 判定 是否有空[] 結果放進新的直欄

for column_name in df.columns:
    df.loc[df[column_name] == 'NaN', column_name] = ''
    df.loc[df[column_name] == '' , column_name] = ''
    df.loc[df[column_name] == '0', column_name] = ''
    df.loc[df[column_name] == 0 , column_name] = ''
    list_index = []
    if type(df[column_name][0]) == type(list_index) :
        df[f"{column_name}_is_emptylist_new"] = False
        for index , element in enumerate(df[column_name]) :
            if len(element) == 0:
                list_index.append(index)
        if len(list_index) > 0:
            df.loc[df.index.isin(list_index),f"{column_name}_is_emptylist_new"] = True
            
# 暫定 => 新增假日欄位 , 預設 False

us_holidays = holidays.country_holidays('US')
us_holidays['2023-1-1']
for holidayname in us_holidays.values() :
    df[holidayname] = False
    
    
# 判定發行商是否是美國 結果放進新的直欄

list_index = []
for index , countrylist in enumerate(df['production_countries']) :
    df['country_is_USA_new'] = False
    if 'United States of America' in countrylist :
        list_index.append(index)
if len(list_index) > 0:
    df.loc[df.index.isin(list_index),"country_is_USA_new"] = True

# movie_id 去重

df = df[df.duplicated(subset='movie_id') == False]

# 篩選預算和票房及發片日期不等於""
# 因為日期有空值,後續會無法處裡是得獎前還是得獎後
# 日期篩選是3/14號開會後才加的判斷式
df_TMDB = df[(df['budget']!="")&(df['revenue']!="")&(df['country_is_USA_new']==True)&(df["release_date"]!="")]


# 存成json檔 方便在家電腦比較慢的可以直接讀取處理好的json檔

df_TMDB.to_json("perfect_movie_release_date_6112")